In [8]:
import os
from typing import List

import geopandas as gpd
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import seaborn as sns
from metr.dataset.interpolator import (Interpolator,
                                       IterativeRandomForestInterpolator,
                                       LinearInterpolator, SplineInterpolator,
                                       TimeMeanFillInterpolator)
from metr.dataset.metr_imc.converter.graph_sensor_locations import \
    SensorNetworkView
from metr.dataset.metr_imc.generator import MetrImcSubsetGenerator
from scipy import stats
from tqdm import tqdm
from metr.components.metr_imc import TrafficData

In [9]:
yn_selected_roads: gpd.GeoDataFrame = gpd.read_file(
    "../datasets/metr-imc-subsets/selected_road.shp"
)
yn_selected_roads

,index,sensor_id,geometry
0,71,1650026001,POINT (126.7067 37.39807)
1,73,1650042800,POINT (126.73509 37.42093)
2,127,1650074500,POINT (126.71392 37.40455)
3,129,1650232500,POINT (126.73677 37.40341)
4,131,1650388801,POINT (126.69317 37.40302)
...,...,...,...
556,16293,1650037302,POINT (126.72667 37.41513)
557,16301,1650002302,POINT (126.71535 37.39645)
558,16311,1650231700,POINT (126.73463 37.40239)
559,16319,1640011900,POINT (126.67742 37.40453)


In [10]:
traffic_data = TrafficData.import_from_hdf("../datasets/metr-imc/metr-imc.h5")
traffic_data.data

2024/09/10 16:53:41 metr.components.metr_imc.traffic_data [INFO] Data sorted by time: h


,1660041701,1610025902,1680058906,1660001801,1660022100,1640014200,1670010800,1610006303,1610005701,1630023200,...,1610002000,1610013813,1663186901,1630047901,1670012500,1650001102,1670153700,1640009500,1650225300,1660030901
2023-09-01 00:00:00,0.0,36.0,98.0,124.0,NaN,33.0,6.0,279.0,325.0,80.0,...,NaN,0.0,99.0,0.0,NaN,63.0,19.0,NaN,0.0,0.0
2023-09-01 01:00:00,0.0,13.0,39.0,74.0,NaN,6.0,0.0,240.0,285.0,37.0,...,NaN,0.0,40.0,0.0,NaN,15.0,0.0,NaN,0.0,0.0
2023-09-01 02:00:00,0.0,18.0,37.0,44.0,NaN,5.0,0.0,235.0,226.0,16.0,...,NaN,0.0,33.0,0.0,NaN,14.0,0.0,NaN,0.0,0.0
2023-09-01 03:00:00,0.0,0.0,21.0,47.0,NaN,0.0,0.0,319.0,150.0,17.0,...,NaN,0.0,13.0,0.0,NaN,41.0,4.0,NaN,0.0,0.0
2023-09-01 04:00:00,0.0,6.0,79.0,141.0,NaN,6.0,0.0,472.0,208.0,53.0,...,NaN,0.0,36.0,0.0,NaN,57.0,15.0,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31 19:00:00,375.0,321.0,268.0,412.0,109.0,365.0,32.0,841.0,805.0,1081.0,...,151.0,157.0,359.0,0.0,NaN,277.0,189.0,10.0,23.0,50.0
2024-08-31 20:00:00,356.0,203.0,208.0,403.0,90.0,296.0,8.0,597.0,923.0,870.0,...,110.0,122.0,348.0,6.0,NaN,204.0,177.0,3.0,10.0,53.0
2024-08-31 21:00:00,289.0,195.0,170.0,333.0,93.0,216.0,13.0,522.0,674.0,808.0,...,79.0,72.0,338.0,0.0,NaN,175.0,96.0,0.0,0.0,22.0
2024-08-31 22:00:00,212.0,116.0,108.0,312.0,63.0,177.0,0.0,526.0,659.0,692.0,...,29.0,40.0,276.0,0.0,NaN,121.0,67.0,0.0,0.0,6.0


In [11]:
target_road_list = yn_selected_roads["sensor_id"].tolist()
temp = ['1630175001', '1630174901', '1630174701', '1630174801', '1640320200', '1630167000', '1630175101', '1650002402', '1650002302']
for item in temp:
    target_road_list.remove(item)
print(target_road_list)
traffic_data.sensor_filter = target_road_list
traffic_data.data

['1650026001', '1650042800', '1650074500', '1650232500', '1650388801', '1650395901', '1650381201', '1640050304', '1640012500', '1640015100', '1640015200', '1630007900', '1640242500', '1650381701', '1640017100', '1650362401', '1650374901', '1640320400', '1650027500', '1650073200', '1640042400', '1640012600', '1650353301', '1640050203', '1640000200', '1640325901', '1640042500', '1650037200', '1650042700', '1650031401', '1640247400', '1640016700', '1640025600', '1640330401', '1640051200', '1640246800', '1640244500', '1640014100', '1650369001', '1640245500', '1640050204', '1640014900', '1650072801', '1630012100', '1640016000', '1640016101', '1650368301', '1650230800', '1610195803', '1650374101', '1640046900', '1650031002', '1650073000', '1650045502', '1640245800', '1650028501', '1650380201', '1640044701', '1640020001', '1630057400', '1640033900', '1650072702', '1640021900', '1640029400', '1630008000', '1640046302', '1650035001', '1650390201', '1650000600', '1650381101', '1650032300', '1640

,1650026001,1650042800,1650074500,1650232500,1650388801,1650395901,1650381201,1640050304,1640012500,1640015100,...,1640245200,1630011803,1640013902,1650352801,1650043801,1640244000,1650037302,1650231700,1640011900,1650090702
2023-09-01 00:00:00,12.0,0.0,NaN,10.0,35.0,0.0,0.0,42.0,0.0,0.0,...,0.0,13.0,3.0,0.0,9.0,0.0,18.0,6.0,136.0,27.0
2023-09-01 01:00:00,6.0,0.0,NaN,0.0,23.0,0.0,0.0,16.0,0.0,0.0,...,0.0,8.0,0.0,0.0,3.0,0.0,4.0,0.0,71.0,13.0
2023-09-01 02:00:00,11.0,0.0,NaN,0.0,6.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8.0,0.0,6.0,0.0,36.0,0.0
2023-09-01 03:00:00,8.0,0.0,NaN,0.0,14.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,42.0,13.0
2023-09-01 04:00:00,27.0,0.0,NaN,0.0,12.0,0.0,0.0,16.0,0.0,0.0,...,3.0,15.0,0.0,0.0,3.0,3.0,0.0,5.0,90.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31 19:00:00,284.0,148.0,12.0,62.0,165.0,146.0,0.0,0.0,0.0,13.0,...,121.0,145.0,186.0,48.0,67.0,121.0,88.0,35.0,225.0,115.0
2024-08-31 20:00:00,248.0,110.0,42.0,67.0,135.0,96.0,0.0,0.0,3.0,25.0,...,98.0,137.0,164.0,44.0,49.0,98.0,89.0,18.0,173.0,119.0
2024-08-31 21:00:00,173.0,94.0,10.0,33.0,114.0,111.0,0.0,0.0,0.0,12.0,...,117.0,89.0,126.0,18.0,27.0,117.0,68.0,17.0,117.0,73.0
2024-08-31 22:00:00,122.0,75.0,6.0,30.0,87.0,39.0,0.0,0.0,0.0,17.0,...,105.0,67.0,72.0,13.0,21.0,105.0,48.0,8.0,51.0,34.0


In [12]:
traffic_data.start_time = "2024-08-15 00:00:00"
traffic_data.data

,1650026001,1650042800,1650074500,1650232500,1650388801,1650395901,1650381201,1640050304,1640012500,1640015100,...,1640245200,1630011803,1640013902,1650352801,1650043801,1640244000,1650037302,1650231700,1640011900,1650090702
2024-08-15 00:00:00,3.0,22.0,0.0,0.0,23.0,15.0,0.0,46.0,0.0,6.0,...,3.0,17.0,20.0,0.0,0.0,3.0,20.0,6.0,12.0,4.0
2024-08-15 01:00:00,5.0,18.0,0.0,0.0,0.0,11.0,0.0,25.0,0.0,6.0,...,0.0,0.0,3.0,0.0,0.0,0.0,15.0,8.0,3.0,0.0
2024-08-15 02:00:00,9.0,6.0,0.0,0.0,10.0,9.0,0.0,6.0,0.0,0.0,...,0.0,4.0,6.0,0.0,0.0,0.0,0.0,0.0,6.0,6.0
2024-08-15 03:00:00,0.0,8.0,0.0,0.0,9.0,4.0,0.0,14.0,0.0,0.0,...,0.0,8.0,6.0,0.0,0.0,0.0,0.0,0.0,6.0,6.0
2024-08-15 04:00:00,15.0,13.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,...,0.0,11.0,3.0,0.0,6.0,0.0,0.0,0.0,12.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31 19:00:00,284.0,148.0,12.0,62.0,165.0,146.0,0.0,0.0,0.0,13.0,...,121.0,145.0,186.0,48.0,67.0,121.0,88.0,35.0,225.0,115.0
2024-08-31 20:00:00,248.0,110.0,42.0,67.0,135.0,96.0,0.0,0.0,3.0,25.0,...,98.0,137.0,164.0,44.0,49.0,98.0,89.0,18.0,173.0,119.0
2024-08-31 21:00:00,173.0,94.0,10.0,33.0,114.0,111.0,0.0,0.0,0.0,12.0,...,117.0,89.0,126.0,18.0,27.0,117.0,68.0,17.0,117.0,73.0
2024-08-31 22:00:00,122.0,75.0,6.0,30.0,87.0,39.0,0.0,0.0,0.0,17.0,...,105.0,67.0,72.0,13.0,21.0,105.0,48.0,8.0,51.0,34.0


In [15]:
traffic_data.end_time = "2024-08-16 23:00:00"
traffic_data.data

,1650026001,1650042800,1650074500,1650232500,1650388801,1650395901,1650381201,1640050304,1640012500,1640015100,...,1640245200,1630011803,1640013902,1650352801,1650043801,1640244000,1650037302,1650231700,1640011900,1650090702
2024-08-15 00:00:00,3.0,22.0,0.0,0.0,23.0,15.0,0.0,46.0,0.0,6.0,...,3.0,17.0,20.0,0.0,0.0,3.0,20.0,6.0,12.0,4.0
2024-08-15 01:00:00,5.0,18.0,0.0,0.0,0.0,11.0,0.0,25.0,0.0,6.0,...,0.0,0.0,3.0,0.0,0.0,0.0,15.0,8.0,3.0,0.0
2024-08-15 02:00:00,9.0,6.0,0.0,0.0,10.0,9.0,0.0,6.0,0.0,0.0,...,0.0,4.0,6.0,0.0,0.0,0.0,0.0,0.0,6.0,6.0
2024-08-15 03:00:00,0.0,8.0,0.0,0.0,9.0,4.0,0.0,14.0,0.0,0.0,...,0.0,8.0,6.0,0.0,0.0,0.0,0.0,0.0,6.0,6.0
2024-08-15 04:00:00,15.0,13.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,...,0.0,11.0,3.0,0.0,6.0,0.0,0.0,0.0,12.0,8.0
2024-08-15 05:00:00,60.0,34.0,0.0,0.0,23.0,9.0,0.0,36.0,0.0,0.0,...,6.0,67.0,17.0,0.0,13.0,6.0,29.0,11.0,42.0,20.0
2024-08-15 06:00:00,79.0,71.0,0.0,0.0,29.0,61.0,0.0,122.0,0.0,0.0,...,0.0,107.0,59.0,0.0,58.0,0.0,29.0,17.0,128.0,51.0
2024-08-15 07:00:00,108.0,87.0,3.0,0.0,46.0,76.0,0.0,140.0,0.0,12.0,...,20.0,132.0,77.0,3.0,47.0,20.0,42.0,35.0,185.0,87.0
2024-08-15 08:00:00,326.0,112.0,0.0,0.0,101.0,111.0,26.0,162.0,0.0,15.0,...,9.0,148.0,99.0,0.0,35.0,9.0,71.0,67.0,166.0,116.0
2024-08-15 09:00:00,390.0,118.0,0.0,0.0,132.0,110.0,26.0,192.0,0.0,33.0,...,44.0,183.0,118.0,15.0,79.0,44.0,73.0,62.0,223.0,117.0
